In [ ]:
import numpy as np
import cv2
import math

In [ ]:
class Prepare:

	def get_playground(self, frame):
		## preprocessing
		rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		lower_red = np.array([70, 0, 0])
		upper_red = np.array([255, 90, 90])
		mask = cv2.inRange(rgb, lower_red, upper_red)
		img_with_mask = cv2.bitwise_and(frame, frame, mask = mask)
		gray_img = cv2.cvtColor(img_with_mask, cv2.COLOR_BGR2GRAY)

		## contour detection
		contours, _ = cv2.findContours(gray_img, 1, 2)

		if not len(contours) > 0:
			raise Exception("Could not find any contour")

		## get second largest square (largest could be border of image)
		index, _ = sorted([[index, cv2.contourArea(cnt)] for index, cnt in enumerate(contours)],
								key=lambda c: c[1], reverse=True)[1]
		cnt = contours[index]

		## define playground borders
		epsilon = 0.01*cv2.arcLength(cnt,True)
		corners = np.squeeze(cv2.approxPolyDP(cnt,epsilon,True), axis=1)

		if not len(corners) == 4:
			raise Exception("could not find four corners of playground")
		
		## get center of playground
		x = [p[0] for p in corners]
		y = [p[1] for p in corners]
		center = (sum(x) / len(corners), sum(y) / len(corners))

		return (corners, center)

In [ ]:
class Prepare:

	def detect_circles(self, img, corners, center, minR_factor, maxR_factor, fields):
		detected_circles, maxNum = np.array([]), 0

		## initialize min and max radius with distance relative to the size of the playground
		minR = round(minR_factor * math.dist(corners[0],center))
		maxR = round(maxR_factor * math.dist(corners[0],center))
		## loop till the minimum radius equals 0 or the required amount of circles is found
		while(minR >= 0):
			print(f"radius: {minR} to {maxR}")
			## detect circles with HoughCircles and specific min and max radius
			circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 50,
				   					   param1 = 120, param2 = 45, 
									   minRadius = minR, maxRadius = maxR)
			if circles is not None:
				numCircles = circles.shape[1]
				print(f"found {numCircles} circles")

				## if the required amount of circles is found, return them
				if numCircles == fields:
					detected_circles = np.squeeze(circles, axis=0)
					break
				## else store as many circle as possible for later evaluation
				elif numCircles > maxNum and numCircles < fields:
					maxNum = numCircles
					detected_circles = np.squeeze(circles, axis=0)
			else:
				print("could not find any circle")

			## if not all required circles were found, decrease the minimum radius
			minR -= 5
		return detected_circles

In [ ]:
class Prepare:

	def identify_green_startingfield(self, frame, street):
		## preprocessing
		imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
		blurred_houses = cv2.medianBlur(imgHSV, 7)
		
		## loop over circles in street list
		for index, (_, x, y, r) in enumerate(street):
			## create empty mask
			mask = np.zeros_like(frame, dtype=np.uint8)
			## draw circle for current field on mask 
			cv2.circle(mask, (int(x), int(y)), int(r), (255,255,255), -1)
			## bitwise_and to leave only pixels inside drawn circle
			mask_area = cv2.bitwise_and(blurred_houses, mask)
			
			## check if pixels in the HSV color range 40-70 (green) are found in masked area
			if self.check_color_in_mask(mask_area, [(40, 100, 100), (70,255,255)]):
				print("finished starting field detection")
				return index
			
		## return -1 if green wasn't detected
		print("could not identify green starting field")
		return -1

	def check_color_in_mask(self, mask, color):
		## search for color with inRange
		lower_color = np.array(color[0], dtype=np.uint8)
		upper_color = np.array(color[1], dtype=np.uint8)
		color_mask = cv2.inRange(mask, lower_color, upper_color)
		## check if any pixels in specified range were found
		return np.sum(color_mask) > 0



In [ ]:
class Ui:
	def __init__(self, BoardCap, DiceCap, GestureCap) -> None:
		## desired size of each frame
		self.shape = (640, 360)

		self.player, self.dice, self.turn, self.prompt, self.movableFigures = "", "", "", "", ""

		## read frame from cv2.VideoCapture and resize
		self.boardFrame = cv2.resize(BoardCap.read()[1], self.shape)
		self.diceFrame = cv2.resize(DiceCap.read()[1], self.shape)
		self.gestureFrame = cv2.resize(GestureCap.read()[1], self.shape)

		## create two horizontal stacks
		## first stacks contains empty frame as background for game status
		overlay = np.zeros((self.shape[1], self.shape[0], 3), np.uint8)
		self.numpy_horizontal_upper = np.hstack((overlay, self.boardFrame))
		self.numpy_horizontal_lower = np.hstack((self.diceFrame, self.gestureFrame))
		## initialize text for game status frame
		self.overlay = self.update_text()
		## stack the horizontal stacks vertically to get 2x2 format
		self.stream = np.vstack((self.numpy_horizontal_upper, self.numpy_horizontal_lower))

	def update(self, overlay=None, boardFrame=None, diceFrame=None, gestureFrame=None):
		## update only frame that was handed over

		## upper horizontal
		if overlay is not None:
			self.overlay = cv2.resize(overlay, self.shape)
			self.numpy_horizontal_upper = np.hstack((self.overlay, self.boardFrame))
		elif boardFrame is not None:
			self.boardFrame = cv2.resize(boardFrame, self.shape)
			self.numpy_horizontal_upper = np.hstack((self.overlay, self.boardFrame))
		## lower horizontal
		elif diceFrame is not None:
			self.diceFrame = cv2.resize(diceFrame, self.shape)
			self.numpy_horizontal_lower = np.hstack((self.diceFrame, self.gestureFrame))
		elif gestureFrame is not None:
			self.gestureFrame = cv2.resize(gestureFrame, self.shape)
			self.numpy_horizontal_lower = np.hstack((self.diceFrame, self.gestureFrame))

		## stack vertically
		self.stream = np.vstack((self.numpy_horizontal_upper, self.numpy_horizontal_lower))
		cv2.imshow("Result", self.stream)

	def update_text(self, player=None, turn=None, dice=None, movableFigures=None, prompt=""):
		## reset current content
		overlay = np.zeros((self.shape[1], self.shape[0], 3), np.uint8)
		
		## fill with new input
		if player is not None:
			self.player = player
		if turn is not None:
			self.turn = turn
		if dice is not None:
			self.dice = dice
		if movableFigures is not None:
			self.movableFigures = movableFigures

		writeList = [
			f"Player: {self.player}",
			f"Current turn: {self.turn}",
			f"Current dice: {self.dice}",
			f"Movable figures: {movableFigures}",
			f"{prompt}"]
					
		for index, entry in enumerate(writeList):
			cv2.putText(overlay, 
					entry,
					(50,50*(index+1)),
					cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
		
		## update current game status with just created one 
		self.update(overlay=overlay)
